In [1]:
import scispacy
import pandas as pd
import re

In [90]:
def explode_protocol_summary(text):
    items = re.findall("<b>(.*?)</b>", text)
    values = re.findall("</b>(.*?)<b>", text)
    protocol = "standard"
    body_part = oral_contrast = iv_contrast = ""
    try:
        if len(values) != len(items):
            values.append(re.findall("(?s:.*)</b>(.*?)$", text)[-1].split("|")[0])
    except:
        pass
    processed_values = []
    for s in values:
        processed_values.append(",".join(string for string in s.split("|") if len(string) > 0))
    if "Protocol:" not in items:
        items.insert(0, "Protocol:")
        processed_values.insert(0, "Standard")
    for i in range(len(items)):
        if items[i] == "Protocol:":
            protocol = processed_values[i]
        if items[i] == "Body Part:":
            body_part = processed_values[i]
        if items[i] == "Oral Contrast:":
            oral_contrast = processed_values[i]
        if items[i] == "IV Contrast:":
            iv_contrast = processed_values[i]
    return (protocol, body_part, oral_contrast, iv_contrast)

def filter_iv_contrast(text):
    if "None" in text:
        return "None"
    elif "Redi-CAT,Rectal" in text:
        return "Redi-CAT,Rectal"
    elif "Omnipaque 300/Water,Rectal" in text:
        return "Omnipaque 300/Water,Rectal"
    elif "Other" in text:
        return "Other"
    elif "Omnipaque 350 - 120 cc" in text:
        return "Omnipaque 350 - 120 cc"
    elif "Omnipaque 350 - 90cc" in text:
        return "Omnipaque 350 - 90 cc"

def filter_oral_contrast(text):
    lst = ["None", "Omnipaque 300/Water", "Volumen", "Water", "Redi-CAT"]
    for l in lst:
        if l in text:
            return l

def filter_body_part(text):
    lst = ["None"]
    for l in lst:
        if l in text:
            return l
    return text

In [91]:
data = pd.read_csv("Past5YCT2.csv", delimiter=",")
filter_rows = ["reasonofstudy", "StudyDesc", "procedurecode", "protocolsummary"]
data = data[filter_rows]

# preprocessing
data = data.dropna()

# drop rows that doesn't have Oral Contrast / IV Contrast
data = data[data["protocolsummary"].str.contains("IV Contrast")]


data["Protocol"], data["Body Part"], data["Oral Contrast"], data["IV Contrast"] = zip(*data.apply(lambda x: explode_protocol_summary(x["protocolsummary"]), axis=1))
data = data[data["IV Contrast"] != ""]
data = data[(data["IV Contrast"] != "") | (data["Oral Contrast"] != "")]
data["IV Contrast"] = data["IV Contrast"].apply(filter_iv_contrast)
data["Oral Contrast"] = data["Oral Contrast"].apply(filter_oral_contrast)
data["Body Part"] = data["Body Part"].apply(filter_body_part)
data = data.dropna()

(91925, 8)

,reasonofstudy,StudyDesc,procedurecode,protocolsummary,Protocol,Oral Contrast,IV Contrast
Body Part,,,,,,,
Abdomen,2765,2765,2765,2765,2765,2765,2765
"Abdomen,Arterial Liver",4,4,4,4,4,4,4
Abdomen/Pelvis,52391,52391,52391,52391,52391,52391,52391
"Abdomen/Pelvis,Arterial Liver",98,98,98,98,98,98,98
"Abdomen/Pelvis,Delayed Kidney",19,19,19,19,19,19,19
Chest/Abdomen/Pelvis,30642,30642,30642,30642,30642,30642,30642
"Chest/Abdomen/Pelvis,Arterial Liver",409,409,409,409,409,409,409
"Chest/Abdomen/Pelvis,Delayed Kidney",16,16,16,16,16,16,16
None,4228,4228,4228,4228,4228,4228,4228


In [70]:
sum(data["Oral Contrast"] == "Redi-CAT,Rectal")

14